In [30]:
from tensorflow.keras.datasets import reuters
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras import layers


In [31]:
# 데이터 로딩 
(train_data, train_labels),(test_data, test_labels) = reuters.load_data(num_words = 10000)

In [32]:
len(train_data),len(test_data)

(8982, 2246)

In [33]:
# 학습데이터에서 첫번째 뉴스 데이터 확인
print (train_data[0])

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [34]:
#
word_index = reuters.get_word_index()
reversed_word_index = dict(
    [(value,key) for (key,value) in word_index.items()]
) 
decoded_newswire = " ".join(
    [reversed_word_index.get(i-3,"?") for i in train_data[0]]
) 

In [35]:
decoded_newswire

'? ? ? said as a result of its december acquisition of space co it expects earnings per share in 1987 of 1 15 to 1 30 dlrs per share up from 70 cts in 1986 the company said pretax net should rise to nine to 10 mln dlrs from six mln dlrs in 1986 and rental operation revenues to 19 to 22 mln dlrs from 12 5 mln dlrs it said cash flow per share this year should be 2 50 to three dlrs reuter 3'

In [36]:
train_labels[27]

1

In [37]:
# 뉴스별 원핫 처리 => 뉴스별 단어 길이를 동일 
def vectorize_seqs(seqs,dims = 10000):
    results = np.zeros((len(seqs),dims))
    for i ,seq in enumerate(seqs):
        for j in seq:
            results[i,j] = 1
    return results

In [38]:
x_train = vectorize_seqs(train_data)
x_test = vectorize_seqs(test_data)

In [39]:
from keras.utils import to_categorical

In [40]:
def to_one_hot(labels,dimension = 46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1
    return results 
y_train = to_one_hot(train_labels)
y_test = to_one_hot(test_labels)

In [41]:
from tensorflow import keras

In [42]:
model = keras.Sequential([
    layers.Dense(64, activation="relu"),
    layers.Dense(64, activation="relu"),
    layers.Dense(46, activation="softmax")
])

In [46]:
model.compile(optimizer="rmsprop",
              loss="categorical_crossentropy",
              metrics=["accuracy"])

In [44]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]
y_val = y_train[:1000]
partial_y_train = y_train[1000:]


In [47]:
history = model.fit(
    partial_x_train
    ,partial_y_train
    ,epochs = 20
    ,batch_size = 512
    ,validation_data = ((x_val,y_val))
)

Epoch 1/20
16/16 [==============================] - 1s 29ms/step - loss: 2.5880 - accuracy: 0.5117 - val_loss: 1.7427 - val_accuracy: 0.6210
Epoch 2/20
16/16 [==============================] - 0s 13ms/step - loss: 1.4912 - accuracy: 0.6810 - val_loss: 1.3480 - val_accuracy: 0.6980
Epoch 3/20
16/16 [==============================] - 0s 13ms/step - loss: 1.1515 - accuracy: 0.7439 - val_loss: 1.1844 - val_accuracy: 0.7390
Epoch 4/20
16/16 [==============================] - 0s 13ms/step - loss: 0.9373 - accuracy: 0.7943 - val_loss: 1.0998 - val_accuracy: 0.7540
Epoch 5/20
16/16 [==============================] - 0s 13ms/step - loss: 0.7708 - accuracy: 0.8365 - val_loss: 0.9817 - val_accuracy: 0.7960
Epoch 6/20
16/16 [==============================] - 0s 14ms/step - loss: 0.6371 - accuracy: 0.8672 - val_loss: 0.9467 - val_accuracy: 0.7900
Epoch 7/20
16/16 [==============================] - 0s 14ms/step - loss: 0.5219 - accuracy: 0.8911 - val_loss: 0.8810 - val_accuracy: 0.8110
Epoch 8/20
16

In [48]:
predictions = model.predict(x_test)

71/71 [==============================] - 0s 3ms/step


In [49]:
np.sum(predictions[0])


0.99999994

In [50]:
np.argmax(predictions[0]) #큰값을 가지고 있는 인덱스 반환

3

In [51]:
predictions[0][3]# 

0.9686525

In [ ]:
# 입력의층이 많이 생기면 힘들다. 신경망을 늘리거나 압축하는것이다.
# 46대로
# 초평면은 3차원이상일때  하이퍼 평면 공간을 압축하려했기 때문에 저차원에 압축하면 픽쳐들의 새로운표현을 하는것너무 줄이는건 안된다.
# 분해를했다가 압축을 조금씩좁혀가란